In [1]:
#System and standard inputs
import sys
import importlib
import timeit
import collections

#Adding correct paths to custom modules
sys.path.append('/Users/sasrey/AlphaTrade/gym_exchange')
sys.path.append('/Users/sasrey/AlphaTrade')
sys.path.append('/Users/sasrey/AlphaTrade/gymnax_exchange')

In [2]:
#Relevant Jax Imports
import jax.numpy as jnp
from jax import lax
import jax

In [134]:
#Direct Import of the Orderbook functions 
import jaxob.JaxOrderbook as job 
#OrderBook class import as written by Kang
from orderbook import OrderBook as cpuOrderBook
#Import (and reload for debugging) of the OrderBook class with jax backend (SF) should behave in the same way.
import jorderbook
importlib.reload(jorderbook)
from jorderbook import OrderBook as JaxOrderBook

#Import Data Mgt and Encoding/Decoding tools
#(The aim of these is to adjust for the case where no message is received but data ends up in the L2 data)
import gym_exchange.data_orderbook_adapter.data_pipeline as dpl
from gym_exchange.data_orderbook_adapter.decoder import Decoder
from gym_exchange.data_orderbook_adapter.encoder import Encoder

In [ ]:
#Attempt at using the data pipeline tools and the encoder/decoder, doesn't really work well. 

dplOBJ=dpl.DataPipeline()
decodeddata=Decoder(10,40,dplOBJ.historical_data,dplOBJ.data_loader)
#Decoder Functionality: Takes the message data (dataloader) and the orderbook data (historical data).
#Creates and maintains an orderbook as implemented in the CPU version.
#Creates and maintains a "DataAdjuster" object, which considers the historical data (orderbook) to be the "true" state
#Inits the orderbook with the first state in the orderbook data (hist) gives imaginary list of IDs to the orders created:
#            each entry in the orderbook is considered to be from a single order. 
output=decodeddata.process()
#Process function for Decoder: goes through N (horizon) messages to ascertain whether there are adidtional messages to consider. 

enc=Encoder(decodeddata)
#enc()

In [245]:
import pandas as pd
df_msg = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_message_50.csv", header=None)
df_ob = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_orderbook_50.csv", header=None)
orderbookLevels=50

In [248]:


data=jnp.array(df_ob.iloc[0,:]).reshape(int(orderbookLevels*2),2)
newarr=jnp.zeros((int(orderbookLevels*2),7))
initOB=newarr.at[:,3].set(data[:,0]).at[:,2].set(data[:,1]).at[:,0].set(1).at[0:orderbookLevels*4:2,1].set(-1).at[1:orderbookLevels*4:2,1].set(1).at[:,4].set(0).at[:,5].set(job.INITID).at[:,6].set(34200.00000000)


df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})
df_orders['TradeID']=0
col_order=['Type','Side','Quantity','Price','TradeID','OrderID','Time']
df_orders=df_orders[col_order]
df_orders_simple=df_orders[(df_orders['Type']!=6)&(df_orders['Type']!=7)&(df_orders['Type']!=5)]

ordersJNP=jnp.array(df_orders)
print(ordersJNP.shape)

(290737, 7)


In [257]:
horizon=200

orderbook=JaxOrderBook(price_levels=50)
trades_init=orderbook.process_orders_array(initOB)
trades_msgs=orderbook.process_orders_array(ordersJNP[0:horizon])

In [259]:
myOB=orderbook.get_L2_state()
L2OB=jnp.array(df_ob.iloc[horizon-1]).astype(float)
L2OB-myOB

Array([ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  2.480e+02,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  6.200e+02,  0.000e+00,  0.000e+00,  0.000e

## Load the original orderbook implementation on the CPU

In [351]:
orderbook_2=cpuOrderBook()

In [353]:
limit_orders = []
order_id_list = [15000000 + i for i in range(20)]
for i in range(10):
    trade_id = 10086
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'ask', 
          'quantity' : df_ob.iloc[0,4* i + 1], 
          'price' : df_ob.iloc[0,4*i],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],
          "timestamp": timestamp}
    limit_orders.append(item)
    
for i in range(10):
    trade_id = 10087
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'bid', 
          'quantity' : df_ob.iloc[0,4* i + 3], 
          'price' : df_ob.iloc[0,4*i+2],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i+10],
          "timestamp": timestamp}
    limit_orders.append(item)
    
# Add init orders to order book
for order in limit_orders:
    # breakpoint()
    trades, order_id = orderbook_2.process_order(order, True, False)  

In [361]:
##Taken largely from the Example.py file in the orderbook repository for the cpu

horizon=50
for index in range(horizon):
    l1 = df_msg.iloc[index,:]
    ttype = l1[1] 
    side = 'bid' if l1[5] ==1 else 'ask'
    quantity = l1[3]
    price = l1[4]
    trade_id = l1[2] # not sure, in the data it is order id
    order_id = trade_id
    timestamp = str(l1[0])
    message = {'type': 'limit','side': side,'quantity': quantity,'price': price,'trade_id': trade_id, "timestamp":timestamp, 'order_id':order_id}
    
    
    best_bid = orderbook_2.get_best_bid()
    
    if ttype == 1:
        message = {'type': 'limit','side': side,'quantity': quantity,'price': price,'trade_id': trade_id, "timestamp":timestamp, 'order_id':order_id}
    elif ttype == 4:##This should be a market order that matches entirely... 
        if side == 'bid' and price <= best_bid:
            side = 'ask'
            message = {'type': 'limit','side': side,'quantity': quantity,'price': price,'trade_id': trade_id, "timestamp":timestamp, 'order_id':order_id}
        else:
            message = None
    elif ttype == 3:
        if price > best_bid: #Why in the world would this be a condition? 3 Should just delete the order in question
            message = None
        else:
            order_book.cancel_order(side, trade_id, time = timestamp)
            message  = None # !remember not to pass the message to be processed
            # breakpoint()
    elif ttype == 6 or ttype == 5:
        message = None
    elif ttype == 2:
        # cancellation (partial deletion of a limit order)
        origin_quantity = order_book.bids.get_order(order_id).quantity # origin_quantity is the quantity in the order book
        adjusted_quantity = origin_quantity - quantity # quantity is the delta quantity
        message = {
            'type' : 'limit',
            'side' : 'bid',
            'quantity': adjusted_quantity,
            'price' : price,
            'order_id': order_id,
            'timestamp': timestamp # the new timestamp
            }
        order_book.bids.update_order(message)
        message = None
    else:
        raise NotImplementedError
        
    if message is not None:
            trades, order_in_book = orderbook_2.process_order(message, True, False)     
        
    

In [362]:
print(orderbook_2)

====================================== Asks ======================================
Quantity   100  |  Price  2585700  |  Order_ID    7381891  |  Time 34200.030920932
Quantity   100  |  Price  2585700  |  Order_ID    7385149  |  Time 34200.041754017
Quantity    10  |  Price  2258800  |  Order_ID      90803  |  Time 34200.214772015
Quantity   600  |  Price  2257800  |  Order_ID    2584622  |  Time 34200.214772015
Quantity    40  |  Price  2257500  |  Order_ID   15000009  |  Time 34200.000000001
Quantity   100  |  Price  2256900  |  Order_ID    2975705  |  Time 34200.214772015
Quantity   100  |  Price  2256000  |  Order_ID    1290262  |  Time 34200.214772015
Quantity    49  |  Price  2255000  |  Order_ID    2584620  |  Time 34200.214772015
Quantity    10  |  Price  2254500  |  Order_ID    5575152  |  Time 34200.214772015
Quantity   100  |  Price  2253900  |  Order_ID    1239894  |  Time 34200.078873974
Quantity    20  |  Price  2252500  |  Order_ID   15000008  |  Time 34200.000000001
Quan